# Cleaner code for generating a new sample, that does not use SDSS-HW redshifts

In [1]:
# imports
from scipy.io import readsav as scirs
import copy, os, glob, imp, pdb
import numpy as np

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Column, Table, QTable
from astropy.io import fits, ascii

from specdb.specdb import IgmSpec, SpecDB
from specdb import query_catalog as spqcat
from specdb import interface_group as spigp
from specdb import utils as spdbu

from linetools import utils as ltu

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.26)

import enigma

## Load newly measured redshifts 

In [2]:
zsys_line = ['H$\\alpha$','H$\\beta$','MgII','[OIII]']
zem_list = []
zem_coord = []
zem_fil = glob.glob(os.getenv('QPQ9')+'Analysis/Redshifts/zem/From_X_Joe/*')
for ff in zem_fil:
    zem = QTable(fits.open(ff)[1].data)
    if zem['ZSYS_FLAG'] not in [0,1,2,11]:
        continue
    zem_list.append(zem)
    ipos = zem['SPEC_FIL'][0].rfind('/SDSSJ')+6
    radec = zem['SPEC_FIL'][0][ipos:]
    ipos2 = radec.rfind('+')
    if ipos2 == -1:
        ipos2 = radec.rfind('-')
    ipos3 = radec.rfind('_F.fits')
    if ipos3 == -1:
        ipos3 = radec.rfind('.fits')
    RA = radec[0:2]+'h'+radec[2:4]+'m'+radec[4:ipos2]+'s'
    dec = radec[ipos2:ipos2+3]+'d'+radec[ipos2+3:ipos2+5]+'m'+radec[ipos2+5:ipos3]+'s'
    zem_coord.append(SkyCoord(RA,dec))
zem_fil = glob.glob(os.getenv('QPQ9')+'Analysis/Redshifts/zem/*_zem.fits')
for ff in zem_fil:
    zem = QTable(fits.open(ff)[1].data)
    if zem['ZSYS_FLAG'] not in [0,1,2,11]:
        continue
    zem_list.append(zem)
    ipos = zem['SPEC_FIL'][0].rfind('/SDSSJ')+6
    radec = zem['SPEC_FIL'][0][ipos:]
    RA = radec[0:2]+'h'+radec[2:4]+'m'+radec[4:9]+'s'
    dec = radec[9:12]+'d'+radec[12:14]+'m'+radec[14:18]+'s'
    zem_coord.append(SkyCoord(RA,dec))
zem_coord = SkyCoord(zem_coord)

## Make a new sample merging QPQ9_zIR and QPQ7

In [3]:
# Fix problems in the QPQ9_zIR structure
zIR_fil = 'qpq9_zIR.fits'
QPQ_zIR = QTable(fits.open(zIR_fil)[1].data)
QPQ9 = copy.deepcopy(QPQ_zIR)
QPQ9.rename_column('FG_ZIR', 'Z_FG')
QPQ9.rename_column('FG_SIG_ZIR','Z_FSIG')
QPQ9.keep_columns(['FG_RA','FG_DEC','NAME','RA','DEC','Z_FG','Z_FSIG','BG_Z','R_PHYS','ZFG_LINE','G_UV'])
for ii,qq in enumerate(QPQ9):
    try:
        assert 'J0225+0048' in qq['NAME'] # redshift is marked to be OII by mistake
        QPQ9.remove_row(ii)
    except AssertionError:
        pass
    try:
        assert 'J0913-0107' in qq['NAME'] # Hb redshift is not good, use MgII
        for zz,zem in enumerate(zem_list):
            if SkyCoord(qq['FG_RA']*u.deg,qq['FG_DEC']*u.deg).separation(zem_coord[zz]) < 1*u.arcsec:
                qq['Z_FG'] = zem['ZSYS_ZSYS'][0]
                qq['Z_FSIG'] = np.rint(zem['ZSYS_ERR'][0])
                qq['ZFG_LINE'] = zsys_line[np.min([zem['ZSYS_FLAG'][0],3])]   
    except AssertionError:
        pass
    try:
        assert 'J1433+0641' in qq['NAME'] # redshift is Null in the zIR structure 
        for zz,zem in enumerate(zem_list):
            if SkyCoord(qq['FG_RA']*u.deg,qq['FG_DEC']*u.deg).separation(zem_coord[zz]) < 1*u.arcsec:
                qq['Z_FG'] = zem['ZSYS_ZSYS'][0]
                qq['Z_FSIG'] = np.rint(zem['ZSYS_ERR'][0])
                qq['ZFG_LINE'] = zsys_line[np.min([zem['ZSYS_FLAG'][0],3])]
    except AssertionError:
        pass
len(QPQ9)

31

In [4]:
# Add QPQ7 pairs with newly measured redshifts 
QPQ7_fil = imp.find_module('enigma')[1] + '/data/qpq/qpq7_pairs.fits.gz'
QPQ7 = QTable(fits.open(QPQ7_fil)[1].data)
c_QPQzIR = SkyCoord(ra=QPQ_zIR['FG_RA']*u.deg, dec=QPQ_zIR['FG_DEC']*u.deg)
c_QPQ7 = SkyCoord(ra=QPQ7['RAD']*u.deg, dec=QPQ7['DECD']*u.deg)
c_QPQ7_bg = SkyCoord(ra=QPQ7['RAD_BG']*u.deg,dec=QPQ7['DECD_BG']*u.deg)
kpc_amin = cosmo.kpc_proper_per_arcmin(QPQ7['Z_FG'])
ang_sep = c_QPQ7.separation(c_QPQ7_bg).to('arcmin')                             
for qq,cc in enumerate(c_QPQ7):
    # do not copy the entry with wrong spectral file 
    wrong_c = SkyCoord(ra=8.59753707028004*u.deg,dec=-10.832307685613417*u.deg)
    if c_QPQ7_bg[qq].separation(wrong_c).to('arcsec') < 0.5*u.arcsec:
        continue
    if len(np.where(cc.separation(c_QPQzIR).to('arcsec') < 0.5*u.arcsec)[0]) == 0:
        if kpc_amin[qq]*ang_sep[qq] < 300*u.kpc:
            for zz,zem in enumerate(zem_list):
                if cc.separation(zem_coord[zz]) < 1*u.arcsec:
                    print(cc.to_string('hmsdms'),'added')
                    QPQ9.add_row({'FG_RA':QPQ7[qq]['RAD'],'FG_DEC':QPQ7[qq]['DECD'],
                                  'NAME':QPQ7[qq]['QSO_BG'],'RA':QPQ7[qq]['RAD_BG'],
                                  'DEC':QPQ7[qq]['DECD_BG'],'Z_FG':zem['ZSYS_ZSYS'][0],
                                  'Z_FSIG':np.rint(zem['ZSYS_ERR'][0]),'BG_Z':QPQ7[qq]['Z_BG'],
                                  'R_PHYS':QPQ7[qq]['R_PHYS'],
                                  'ZFG_LINE':zsys_line[np.min([zem['ZSYS_FLAG'][0],3])],
                                  'G_UV':QPQ7[qq]['G_UV']})
                        
QPQ9.sort(['FG_RA','FG_DEC'])

(u'12h31m43.0985s +00d28m46.2929s', 'added')
(u'14h09m18.0061s +52d25m52.3684s', 'added')
(u'14h44m29.3999s +31d13m21.2384s', 'added')
(u'16h05m47.6055s +51d13m30.5441s', 'added')


In [5]:
len(QPQ9),QPQ9['NAME','ZFG_LINE'][20:34]

(35, <QTable length=14>
      NAME       ZFG_LINE
     str14        str36  
 -------------- ---------
 SDSSJ1301+4759 H$\alpha$
  APOJ1306+6158 H$\alpha$
 BOSSJ1402+4701    [OIII]
 SDSSJ1409+5225      MgII
 BOSSJ1420+0227    [OIII]
  APOJ1420+1603    [OIII]
 BOSSJ1427-0121    [OIII]
 BOSSJ1433+0641    [OIII]
 APOJ1444+3113       MgII
  APOJ1553+1921    [OIII]
 APOJ1605+5113       MgII
 BOSSJ1627+2215  H$\beta$
  APOJ1627+4606    [OIII]
  APOJ2146-0753    [OIII])

In [6]:
# Add QPQ7 pairs with MgII redshifts
c_QPQ9 = SkyCoord(ra=QPQ9['FG_RA']*u.deg, dec=QPQ9['FG_DEC']*u.deg)
for qq,cc in enumerate(c_QPQ7):
    # do not copy the entry with wrong spectral file 
    wrong_c = SkyCoord(ra=8.59753707028004*u.deg,dec=-10.832307685613417*u.deg)
    if c_QPQ7_bg[qq].separation(wrong_c).to('arcsec') < 0.5*u.arcsec:
        continue
    if len(np.where(cc.separation(c_QPQ9).to('arcsec') < 1*u.arcsec)[0]) == 0:
        if kpc_amin[qq]*ang_sep[qq] < 300*u.kpc:
            if QPQ7['Z_FSIG'][qq] < 300:
                QPQ9.add_row({'FG_RA':QPQ7['RAD'][qq],'FG_DEC':QPQ7['DECD'][qq],
                              'NAME':QPQ7['QSO_BG'][qq],'RA':QPQ7['RAD_BG'][qq],
                              'DEC':QPQ7['DECD_BG'][qq],'Z_FG':QPQ7['Z_FG'][qq],
                              'Z_FSIG':272,'BG_Z':QPQ7['Z_BG'][qq],
                              'R_PHYS':QPQ7['R_PHYS'][qq],'ZFG_LINE':'MgII',
                              'G_UV':QPQ7['G_UV'][qq]})

QPQ9.sort(['FG_RA','FG_DEC'])

In [7]:
len(QPQ9),QPQ9['NAME'][0:20]

(59, <Column name='NAME' dtype='str14' length=20>
 BOSSJ0057-0001
 SDSSJ0149-0021
 BOSSJ0230-0333
 SDSSJ0341-0000
 SDSSJ0409-0411
 SDSSJ0752+4011
  APOJ0800+3542
 SDSSJ0828+4545
 APOJ0837+3630 
  APOJ0837+3837
 BOSSJ0841+3921
 BOSSJ0853-0011
  APOJ0910+0414
 BOSSJ0913-0107
 BOSSJ0914+0109
 APOJ0924+3929 
  APOJ0932+0925
 BOSSJ0936-0058
  APOJ0938+5317
 BOSSJ1002+0020)

In [8]:
# write
Table(QPQ9).write('qpq9_final.fits',format='fits',overwrite=True)

## Add pairs in QPQ database with newly measured redshifts 

In [9]:
QPQ9 = QTable(fits.open('qpq9_final.fits')[1].data)
c_QPQ9 = SkyCoord(QPQ9['FG_RA']*u.deg,QPQ9['FG_DEC']*u.deg)
c_QPQ9_bg = SkyCoord(ra=QPQ9['RA']*u.deg,dec=QPQ9['DEC']*u.deg)
qpq_fil = '/Users/lwymarie/Documents/Databases/qpq_oir_spec.hdf5'
qpqsp = SpecDB(db_file=qpq_fil,verbose=False,idkey='QPQ_ID')
ID_fg,ID_bg = qpqsp.qcat.pairs(0.92*u.arcmin, 3000.*u.km/u.s)
c_qpqsp = SkyCoord(qpqsp.cat['RA'][ID_fg]*u.deg,qpqsp.cat['DEC'][ID_fg]*u.deg)
c_qpqsp_bg = SkyCoord(qpqsp.cat['RA'][ID_bg]*u.deg,qpqsp.cat['DEC'][ID_bg]*u.deg)
kpc_amin = cosmo.kpc_proper_per_arcmin(qpqsp.cat['zem'][ID_fg])
ang_sep = c_qpqsp.separation(c_qpqsp_bg).to('arcmin')
for qq,cc in enumerate(c_qpqsp):
    if kpc_amin[qq]*ang_sep[qq] < 300 *u.kpc: 
        if len(np.where(cc.separation(c_QPQ9) < 0.5*u.arcsec)[0]) == 0: # pair not in QPQ9 already
            if len(np.where(cc.separation(zem_coord) < 1*u.arcsec)[0]) > 0:
                #J0225 has no good line measured. Skip.
                if cc.separation(SkyCoord('02h25m17.68s +00d48m22s')) < 1*u.arcsec:
                    continue
                #J0239 add [OIII]. Did not pass Lyb cut. Add back. 
                #J1112 has good Hb, but currently it's MgII from QPQ7. 
                #The redshift from QPQ7 or Hb here does not pass the > 3000 km/s requirement. Skip. 
                if cc.separation(SkyCoord('11h12m42.69s +66d11m52.8s')) < 1*u.arcsec:
                    continue
                #J1215 add [OIII].
                #J2338 add [OIII]. 
                name = 'BOSSJ'
                name = name + c_qpqsp_bg[qq].to_string('hmsdms')[0:2] + c_qpqsp_bg[qq].to_string('hmsdms')[3:5]
                ipos = c_qpqsp_bg[qq].to_string('hmsdms').rfind(' ')+1
                name = name + c_qpqsp_bg[qq].to_string('hmsdms')[ipos:ipos+3] + \
                c_qpqsp_bg[qq].to_string('hmsdms')[ipos+4:ipos+6]
                index = np.where(zem_coord.separation(cc) < 1*u.arcsec)[0]
                if len(index) == 0:
                    print('no match found for',cc.to_string('hmsdms'),c_qpqsp_bg[qq].to_string('hmsdms'))
                    continue
                QPQ9.add_row({'FG_RA':qpqsp.cat[ID_fg[qq]]['RA'],'FG_DEC':qpqsp.cat[ID_fg[qq]]['DEC'],
                              'NAME':name,'RA':qpqsp.cat[ID_bg[qq]]['RA'],'DEC':qpqsp.cat[ID_bg[qq]]['DEC'],
                              'Z_FG':zem_list[index[0]]['ZSYS_ZSYS'][0],
                              'Z_FSIG':np.rint(zem_list[index[0]]['ZSYS_ERR'][0]),
                              'BG_Z':qpqsp.cat[ID_bg[qq]]['zem'],'R_PHYS':(kpc_amin[qq]*ang_sep[qq]).value,
                              'ZFG_LINE':zsys_line[np.min([zem_list[index[0]]['ZSYS_FLAG'][0],3])]})
                print(cc.to_string('hmsdms'),'added')
QPQ9.sort(['FG_RA','FG_DEC'])

Database is QPQ
Created on 2017-Jan-31
(u'08h14m19.58s +32d50m18.7s', 'added')
(u'09h25m43.8s +37d25m04.9s', 'added')
(u'02h39m46.43s -01d06m40.4s', 'added')
(u'00h34m23.05s -10d50m02s', 'added')
(u'02h50m38.68s -00d47m39.2s', 'added')
(u'07h50m09.25s +27d24m05.2s', 'added')
(u'10h19m47.1s +49d48m35.8s', 'added')
(u'12h15m59s +57d16m16.6s', 'added')


In [10]:
len(QPQ9),QPQ9[0:20]

(67, <QTable length=20>
       RA             DEC             G_UV     ...     R_PHYS     ZFG_LINE
    float64         float64         float64    ...    float64      str36  
 ------------- ----------------- ------------- ... ------------- ---------
        8.5975    -10.8323055556           0.0 ... 67.1230838982      MgII
 14.3287633684  -0.0263108702349 897.407834846 ... 270.712499511    [OIII]
 27.3227662484   -0.366242429577 1726.12219238 ... 154.683044434      MgII
 37.5832882888    -3.55444111362 1011.23883057 ... 220.653686523      MgII
 39.9435416667          -1.11225           0.0 ... 31.6279553361    [OIII]
 42.6659166667   -0.797111111111           0.0 ...  174.68978497      MgII
 55.4133096521 -0.00353495681141 274.194061279 ... 190.162078857      MgII
 62.4758752949    -4.19363997107 515.653686523 ... 235.123641968      MgII
 117.534458333          27.40125           0.0 ... 114.302432635      MgII
 118.246405235     40.1884027001 749.980773926 ... 110.227485657      MgII
 

In [11]:
# write
Table(QPQ9).write('qpq9_final.fits',format='fits',overwrite=True)

## Add pairs in igmspec with newly measured redshifts

In [12]:
QPQ9 = QTable(fits.open('qpq9_final.fits')[1].data)
c_QPQ9 = SkyCoord(QPQ9['FG_RA']*u.deg,QPQ9['FG_DEC']*u.deg)
c_QPQ9_bg = SkyCoord(ra=QPQ9['RA']*u.deg,dec=QPQ9['DEC']*u.deg)
igmsp = IgmSpec(version="02.1",verbose=False,
                groups=['BOSS_DR12','COS-Dwarfs','COS-Halos',
                        'ESI_DLA','GGG','HD-LLS_DR1',
                        'HDLA100','HSTQSO','HST_z2',
                        'KODIAQ_DR1','SDSS_DR7','XQ-100',
                        'UVES_Dall','UVpSM4','MUSoDLA'])
ID_fg,ID_bg = igmsp.qcat.pairs(0.92*u.arcmin,3000.*u.km/u.s) # at z = 0.4, 0.92' = 300 kpc
c_igmsp = SkyCoord(igmsp.cat['RA'][ID_fg]*u.deg,igmsp.cat['DEC'][ID_fg]*u.deg)
c_igmsp_bg = SkyCoord(igmsp.cat['RA'][ID_bg]*u.deg,igmsp.cat['DEC'][ID_bg]*u.deg)
kpc_amin = cosmo.kpc_proper_per_arcmin(igmsp.cat['zem'][ID_fg])
ang_sep = c_igmsp.separation(c_igmsp_bg).to('arcmin')
for qq,cc in enumerate(c_igmsp):
    if kpc_amin[qq]*ang_sep[qq] < 300 *u.kpc: 
        if len(np.where(cc.separation(c_QPQ9) < 0.5*u.arcsec)[0]) == 0: # pair not in QPQ9 already
            if len(np.where(cc.separation(zem_coord) < 1*u.arcsec)[0]) > 0:
                # Skip J0225. No good line measured. 
                if cc.separation(SkyCoord('02h25m17.68s +00d48m22s')) < 5*u.arcsec:
                    continue
                # Skip J0201. Binary quasar, with wrong redshift in catalog. 
                if cc.separation(SkyCoord('02h01m43.4873s +00d32m22.713s')) < 5*u.arcsec:
                    continue
                name = 'SDSSJ'
                name = name + c_igmsp_bg[qq].to_string('hmsdms')[0:2] + c_igmsp_bg[qq].to_string('hmsdms')[3:5]
                ipos = c_igmsp_bg[qq].to_string('hmsdms').rfind(' ')+1
                name = name + c_igmsp_bg[qq].to_string('hmsdms')[ipos:ipos+3] + \
                c_igmsp_bg[qq].to_string('hmsdms')[ipos+4:ipos+6]
                index = np.where(zem_coord.separation(cc) < 1*u.arcsec)[0]
                QPQ9.add_row({'FG_RA':igmsp.cat[ID_fg[qq]]['RA'],'FG_DEC':igmsp.cat[ID_fg[qq]]['DEC'],
                              'NAME':name,'RA':igmsp.cat[ID_bg[qq]]['RA'],'DEC':igmsp.cat[ID_bg[qq]]['DEC'],
                              'Z_FG':zem_list[index[0]]['ZSYS_ZSYS'][0],
                              'Z_FSIG':np.rint(zem_list[index[0]]['ZSYS_ERR'][0]),'BG_Z':igmsp.cat[ID_bg[qq]]['zem'],
                              'R_PHYS':(kpc_amin[qq]*ang_sep[qq]).value,
                              'ZFG_LINE':zsys_line[np.min([zem_list[index[0]]['ZSYS_FLAG'][0],3])]})
                print(cc.to_string('hmsdms'),'added')
QPQ9.sort(['FG_RA','FG_DEC'])

Loading igmspec from /Users/lwymarie/Documents/Databases/IGMspec_DB_v02.1.hdf5
Database is igmspec
Created on 2017-Sep-18
(u'00h42m20.6566s +00d32m18.6839s', 'added')
(u'00h47m45.4931s +31d01m20.2527s', 'added')
(u'01h22m28.0063s +30d17m34.7976s', 'added')
(u'01h43m28.7665s +29d54m36.8018s', 'added')
(u'02h25m26.3271s -01d14m34.3976s', 'added')
(u'02h26m30.7915s -05d05m52.215s', 'added')
(u'02h32m26.5389s -00d11m45.1292s', 'added')
(u'02h33m15.4029s -00d03m03.5561s', 'added')
(u'02h46m03.6827s -00d32m11.7543s', 'added')
(u'07h27m39.5462s +39d28m55.3406s', 'added')
(u'08h05m37.2907s +47d23m39.3058s', 'added')
(u'08h09m38.7284s +50d09m47.7073s', 'added')
(u'08h23m07.087s +53d47m38.3889s', 'added')
(u'08h23m46.0482s +53d25m27.8191s', 'added')
(u'08h24m21.0108s +53d12m49.2937s', 'added')
(u'08h35m26.7843s +46d20m07.4548s', 'added')
(u'08h43m38.8552s +44d26m08.6726s', 'added')
(u'08h45m11.8996s +46d41m35.5249s', 'added')
(u'08h46m18.1828s +45d39m00.306s', 'added')
(u'08h50m19.4257s +47d55m3

In [13]:
len(QPQ9),QPQ9['NAME','FG_RA','FG_DEC'][180:200]

(210, <QTable length=20>
      NAME          FG_RA         FG_DEC   
     str14         float64       float64   
 -------------- ------------- -------------
 BOSSJ1508+3635 227.058590007 36.5914880608
 SDSSJ1516+2225 229.047900888 22.4209050325
 BOSSJ1533+1425 233.370129953 14.4284725638
 SDSSJ1534+2153    233.733417     21.895089
 SDSSJ1539+3146      234.9781     31.774801
 SDSSJ1541+4919 235.249450505 49.3231468351
 SDSSJ1543+4445 235.954654103 44.7540928733
 SDSSJ1544+5027    236.089179     50.446078
 SDSSJ1548+5459 237.068459669 54.9949789692
  APOJ1553+1921 238.356698707 19.3613755865
 SDSSJ1554+1244    238.595322     12.743891
 SDSSJ1557+2929 239.391580654 29.4789835582
  APOJ1559+4943 239.948892303 49.7186922034
  APOJ1605+5113 241.448356284 51.2251511404
 SDSSJ1619+1926    244.878904     19.439138
 BOSSJ1627+2215  246.90165871  22.260579174
  APOJ1627+4606 246.910970979 46.0939966094
 SDSSJ1631+4333    247.840601     43.554805
 SDSSJ1654+2513 253.675865835 25.2136755441
 BOSSJ1

In [14]:
# write
Table(QPQ9).write('qpq9_final.fits',format='fits',overwrite=True)

## Add pairs of z_fg < 1.6 and measured by SDSS-HW 

In [15]:
QPQ9 = QTable(fits.open('qpq9_final.fits')[1].data)
c_QPQ9 = SkyCoord(QPQ9['FG_RA']*u.deg,QPQ9['FG_DEC']*u.deg)
c_QPQ9_bg = SkyCoord(ra=QPQ9['RA']*u.deg,dec=QPQ9['DEC']*u.deg)
for qq,cc in enumerate(c_igmsp):
    if kpc_amin[qq]*ang_sep[qq] < 300 *u.kpc: 
        if 'SDSS-HW' in igmsp.cat[ID_fg[qq]]['flag_zem']: # good z
            # Match Johnson+15 and Farina+ samples' redshifts, and MgII redshift or better 
            if (igmsp.cat[ID_fg[qq]]['zem'] >= 0.4) & (igmsp.cat[ID_fg[qq]]['zem'] <= 1.6): 
                # Lya forest cut for MgII stack 
                if 2796.354*(1+igmsp.cat['zem'][ID_fg[qq]]) > (1215.6701+15.)*(1+igmsp.cat['zem'][ID_bg[qq]]):
                    if len(np.where(cc.separation(c_QPQ9) < 0.5*u.arcsec)[0]) > 0: # in QPQ9 already
                        continue
                    if np.mod(igmsp.cat['flag_group'][ID_fg[qq]],2) == 0:
                        name = 'BOSSJ'
                    else:
                        name = 'SDSSJ'  
                    name = name + c_igmsp_bg[qq].to_string('hmsdms')[0:2]
                    name = name + c_igmsp_bg[qq].to_string('hmsdms')[3:5]
                    ipos = c_igmsp_bg[qq].to_string('hmsdms').rfind(' ')+1
                    name = name + c_igmsp_bg[qq].to_string('hmsdms')[ipos:ipos+3] 
                    name = name + c_igmsp_bg[qq].to_string('hmsdms')[ipos+4:ipos+6]
                    QPQ9.add_row({'FG_RA':igmsp.cat['RA'][ID_fg[qq]],'FG_DEC':igmsp.cat['DEC'][ID_fg[qq]],'NAME':name,
                                  'RA':igmsp.cat['RA'][ID_bg[qq]],'DEC':igmsp.cat['DEC'][ID_bg[qq]],
                                  'Z_FG':igmsp.cat['zem'][ID_fg[qq]],'Z_FSIG':272,'BG_Z':igmsp.cat['zem'][ID_bg[qq]],
                                  'R_PHYS':(kpc_amin[qq]*ang_sep[qq]).value,'ZFG_LINE':'SDSS-HW'})
QPQ9.sort(['FG_RA','FG_DEC'])

In [16]:
len(QPQ9),QPQ9['NAME','Z_FG','ZFG_LINE','FG_RA','FG_DEC'][580:583]

(670, <QTable length=3>
      NAME           Z_FG     ZFG_LINE     FG_RA         FG_DEC   
     str14         float64     str36      float64       float64   
 -------------- ------------- -------- ------------- -------------
 SDSSJ1543+4445 1.85299101525     MgII 235.954654103 44.7540928733
 SDSSJ1544+5027 1.60675870161     MgII    236.089179     50.446078
 BOSSJ1545+2239 1.00233602524  SDSS-HW    236.472613     22.669987)

In [17]:
# write
Table(QPQ9).write('qpq9_final.fits',format='fits',overwrite=True)

## Add and update g_UV values

In [19]:
# Search SDSS DR12 catalog for magnitudes 
QPQ9 = QTable(fits.open('qpq9_final.fits')[1].data)
c_QPQ9 = SkyCoord(QPQ9['FG_RA']*u.deg,QPQ9['FG_DEC']*u.deg)
c_QPQ9_bg = SkyCoord(ra=QPQ9['RA']*u.deg,dec=QPQ9['DEC']*u.deg)
hdulist = fits.open('/Volumes/Data/Data of surveys/SDSS/DR12Q.fits')
SDSSdr12 = hdulist[1].data
c_SDSSdr12 = SkyCoord(list(SDSSdr12.field('RA'))*u.deg,list(SDSSdr12.field('DEC'))*u.deg)
hdulist = fits.open('/Volumes/Data/Data of surveys/SDSS/dr7qso.fit')
SDSSdr7 = hdulist[1].data
c_SDSSdr7 = SkyCoord(list(SDSSdr7.field('RA'))*u.deg,list(SDSSdr7.field('DEC'))*u.deg)

In [20]:
# Put in g_UV by hand for a quasar that does not have SDSS spectra but photometry and no existing g_UV values 
for qq,cc in enumerate(c_QPQ9):
    if QPQ9[qq]['G_UV'] == 0.:
        if len(np.where(cc.separation(c_SDSSdr12) < 0.5*u.arcsec)[0]) == 0:
            if len(np.where(cc.separation(c_SDSSdr7) < 0.5*u.arcsec)[0]) == 0:
                ang_sep = cc.separation(c_QPQ9_bg[qq]).to('arcsec')
                print(cc.to_string('hmsdms'),QPQ9[qq]['Z_FG'],ang_sep,qq)
                QPQ9[qq]['G_UV'] = 4624

(u'01h11m50.07s +14d01m41.5s', 2.4851422501937015, <Angle 35.64055373774893 arcsec>, 30)


In [21]:
# Write a file for IDL input, for rescaling QPQ g_UV values to F.-G. Lya forest flux 
gUV_exist = np.where(QPQ9['G_UV'] != 0.)[0]
data = Table([QPQ9[gUV_exist]['FG_RA'],QPQ9[gUV_exist]['FG_DEC'],QPQ9[gUV_exist]['Z_FG'],QPQ9[gUV_exist]['G_UV']],
             names=['FG_RA','FG_DEC','Z_FG','OLD_GUV'])
ascii.write(data,'QPQ_gUV_idl_input.txt',overwrite=True)
# Write a file for IDL input, for calculating new g_UV values 
gUV_none = np.where(QPQ9['G_UV'] == 0.)[0]
ang_sep = []
SDSSfilter = []
mag = []
filter_beg = [3125.0, 3880.0, 5480.0, 6790.0, 8090.0]
filter_names = ['u','g','r','i','z']
for qq,cc in enumerate(c_QPQ9[gUV_none]):
    ang_sep.append(cc.separation(c_QPQ9_bg[gUV_none[qq]]).to('arcsec').value)
    min_beg = 1215.6701*(1+QPQ9[gUV_none[qq]]['Z_FG'])
    good_inds = np.where(filter_beg >= min_beg)
    SDSSfilter.append(filter_names[np.min(good_inds)])
    in_dr12 = np.where(cc.separation(c_SDSSdr12) < 0.5*u.arcsec)[0]
    if len(in_dr12) == 1:
        mag.append((SDSSdr12[in_dr12].field('PSFMAG')[0][good_inds]-
                    SDSSdr12[in_dr12].field('EXTINCTION')[0][good_inds])[0])
    else:
        in_dr7 = np.where(cc.separation(c_SDSSdr7) < 0.5*u.arcsec)[0]
        psfmag = [SDSSdr7[in_dr7].field('UMAG'),SDSSdr7[in_dr7].field('GMAG'),SDSSdr7[in_dr7].field('RMAG'),
                  SDSSdr7[in_dr7].field('IMAG'),SDSSdr7[in_dr7].field('ZMAG')]
        extinct = SDSSdr7[in_dr7].field('AU')*np.array([1,0.736,0.534,0.405,0.287])
        mag.append((psfmag[np.min(good_inds)]-extinct[np.min(good_inds)])[0])
data = Table([QPQ9[gUV_none]['FG_RA'],QPQ9[gUV_none]['FG_DEC'],QPQ9[gUV_none]['Z_FG'],ang_sep,SDSSfilter,mag],
             names=['FG_RA','FG_DEC','Z_FG','theta','filter','mag'])
ascii.write(data,'no_gUV_idl_input.txt',overwrite=True)

In [22]:
# Go to run IDL

# Read IDL output for rescaled QPQ g_UV values 
data = ascii.read('QPQ_gUV_idl_output.txt')
c_idl = SkyCoord(data['FG_RA']*u.deg,data['FG_DEC']*u.deg)
for qq,cc in enumerate(c_QPQ9):
    index = np.where(cc.separation(c_idl) < 0.5*u.arcsec)[0]
    if len(index) == 1:
        QPQ9[qq]['G_UV'] = data[index]['NEW_GUV']
# Read IDL output for new g_UV values
data = ascii.read('no_gUV_idl_output.txt')
c_idl = SkyCoord(data['FG_RA']*u.deg,data['FG_DEC']*u.deg)
for qq,cc in enumerate(c_QPQ9):
    index = np.where(cc.separation(c_idl) < 0.5*u.arcsec)[0]
    if len(index) == 1:
        QPQ9[qq]['G_UV'] = data[index]['GUV']

In [23]:
np.where(QPQ9['G_UV'] == 0.)

(array([], dtype=int64),)

In [24]:
QPQ9['FG_RA','FG_DEC'][580:583]

FG_RA,FG_DEC
float64,float64
235.954654103,44.7540928733
236.089179,50.446078
236.472613,22.669987


In [25]:
# write
Table(QPQ9).write('qpq9_final.fits',format='fits',overwrite=True)

## Update systematic offsets and scatters of emission lines to Shen+16 

In [26]:
# Add the systematic biases, intrinsic scatter + measurement scatter
QPQ9 = QTable(fits.open('qpq9_final.fits')[1].data)
for ii,qq in enumerate(QPQ9):
    if qq['ZFG_LINE'] == 'MgII':
        # bring to z_em 
        qq['Z_FG'] = ltu.z_from_dv(+70.*u.km/u.s,qq['Z_FG'])
        # bring to z_sys
        qq['Z_FG'] = ltu.z_from_dv(+57.*u.km/u.s,qq['Z_FG'])
        # scatter
        qq['Z_FSIG'] = 226
    if qq['ZFG_LINE'] == 'H$\\beta$':
        # bring to z_sys
        qq['Z_FG'] = ltu.z_from_dv(+109.*u.km/u.s,qq['Z_FG'])
        # scatter 
        qq['Z_FSIG'] = 417
    if qq['ZFG_LINE'] == '[OIII]':
        # bring to z_em 
        qq['Z_FG'] = ltu.z_from_dv(-27.*u.km/u.s,qq['Z_FG'])
        # bring to z_sys
        qq['Z_FG'] = ltu.z_from_dv(+48.*u.km/u.s,qq['Z_FG'])
        # scatter
        qq['Z_FSIG'] = 68
    if qq['ZFG_LINE'] == 'H$\\alpha$':
        # scatter
        qq['Z_FSIG'] = 300
    if qq['ZFG_LINE'] == 'SDSS-HW':
        if qq['Z_FG'] >= 0.84:
            # MgII redshift, bring to z_sys
            qq['Z_FG'] = ltu.z_from_dv(+57.*u.km/u.s,qq['Z_FG'])
            # scatter
            qq['Z_FSIG'] = 226
        else:
            # [OIII] redshift, bring to zsys
            qq['Z_FG'] = ltu.z_from_dv(+48.*u.km/u.s,qq['Z_FG'])
            # scatter
            qq['Z_FSIG'] = 68

In [27]:
# write
Table(QPQ9).write('qpq9_final.fits',format='fits',overwrite=True)

## copy to enigma

In [28]:
!cp qpq9_final.fits /Users/lwymarie/python/enigma/enigma/data/qpq/qpq9_final.fits
#!cp /Users/lwymarie/python/enigma/enigma/data/qpq/qpq9_final.fits qpq_final.fits
len(QPQ9)

670

In [29]:
QPQ9 = QTable(fits.open('qpq9_final.fits')[1].data)
QPQ9['NAME','RA','DEC','Z_FG','BG_Z'][200:220]

NAME,RA,DEC,Z_FG,BG_Z
str14,float64,float64,float64,float64
SDSSJ0939+4820,144.912373128,48.3498439811,1.88780396121,2.41459201199
SDSSJ0939+5051,144.97813827,50.8635518455,1.61045174969,2.47634572075
BOSSJ0941+3058,145.305848815,30.9673281517,1.19488029089,2.73344566311
BOSSJ0941+3251,145.371275774,32.8619651249,1.06858622492,2.38920955929
SDSSJ0941+2308,145.398374434,23.146068427,1.77619046657,2.55107683578
BOSSJ0942+0716,145.574568511,7.26677619945,0.803799808173,2.54694621275
BOSSJ0943+5842,145.857402363,58.7077452527,0.757433382413,2.01750978774
BOSSJ0944+4807,146.031517132,48.1305794445,1.11508013045,2.35010844785
BOSSJ0944+2321,146.05431893,23.3612505552,1.52541905716,2.98069514796


In [49]:
1334.5323*(1+1.741),1334.5323*(1+ltu.z_from_dv(-3000*u.km/u.s,1.741)),1334.5323*(1+ltu.z_from_dv(3000*u.km/u.s,1.741))

(3657.9530343, 3621.5295122904486, 3694.7428857709247)

In [51]:
qpq_fil = '/Users/lwymarie/Documents/Databases/qpq_oir_spec.hdf5'
qpqsp = SpecDB(db_file=qpq_fil,verbose=False,idkey='QPQ_ID')
igmsp = IgmSpec(version="02.1",verbose=False,
                groups=['BOSS_DR12','COS-Dwarfs','COS-Halos',
                        'ESI_DLA','GGG','HD-LLS_DR1',
                        'HDLA100','HSTQSO','HST_z2',
                        'KODIAQ_DR1','SDSS_DR7','XQ-100',
                        'UVES_Dall','UVpSM4','MUSoDLA'])
specs,metas = igmsp.spectra_from_coord(SkyCoord(147.859266011*u.deg,49.5385579181*u.deg),tol=1*u.arcsec)
metas
specs[0].plot()

Database is QPQ
Created on 2017-Jan-31
Loading igmspec from /Users/lwymarie/Documents/Databases/IGMspec_DB_v02.1.hdf5
Database is igmspec
Created on 2017-Sep-18
Your search yielded 1 match[es] within radius=1 arcsec
Staged 1 spectra totalling 9.3e-05 Gb
Loaded spectra
Staged 1 spectra totalling 8e-05 Gb
Loaded spectra
